# Amazon Rekognition vs Dlib: Face Detection Comparison

In [22]:
import dlib
import boto3
import botocore
import numpy as np
import cv2
from os import getcwd, listdir
import dlib
import time

PROFILE = "innovacion-dev-developer"
BUCKET = "poc-images-rekognition"
DATA_FOLDER = "../Data/Detection/"
SAVE_FOLDER = DATA_FOLDER + "Results/"
REGION = "eu-west-1"
WITH_PROFILE = 1
S3_FOLDER = "Detection/"
CNN_MODEL_PATH = "../Dlib/Models/mmod_human_face_detector.dat"
CNN = 0

if CNN:
    tag = "/cnn_"
else:
    tag = "/hog_"

In [23]:
if WITH_PROFILE:
    session = boto3.Session(profile_name=PROFILE)
    s3 = session.client("s3")
    rekognition = session.client("rekognition", REGION)
else:
    s3 = boto3.client("s3")
    rekognition = boto3.client("rekognition", REGION)

In [24]:
files = listdir(LOCAL_FOLDER, )
images_names = sorted([f for f in files if f.endswith('.jpg')])
images = [cv2.imread(LOCAL_FOLDER + image) for image in images_names]
n_images = len(images)

In [25]:
def dlib_detect_faces(image, cnn = 0):
    
    if CNN:
        detector = dlib.cnn_face_detection_model_v1(CNN_MODEL_PATH)
    else:
        detector = dlib.get_frontal_face_detector()
    
    dets = detector(image, 1)
    return dets

In [26]:
def aws_detect_faces(bucket, image, rekgonition_client):
    response = rekgonition_client.detect_faces(
        Image =
        {
            "S3Object": 
            {
                "Bucket": bucket,
                "Name": image,
            }
        }
    )
    return response

In [27]:
def create_rectangle(coordinates, image_heigth, image_width):
    bounding_check = dlib.rectangles()
    for i in range(0, len(coordinates)):
        coord = coordinates[i]
        left = int(coord['Left']*image_width)
        width = int(coord['Width']*image_width)
        top = int(coord['Top']*image_heigth)
        heigth = int(coord['Height']*image_heigth)
        rect = dlib.rectangle(left, top, left + width, top + heigth)
        bounding_check.append(rect)
    return bounding_check

In [28]:
dlib_times = list()
aws_times = list()

# win = dlib.image_window()

for i in range(0, n_images):
    img = images[i]
    img_name = images_names[i]
    
    print("{}. Processing image: {}".format(i+1, S3_FOLDER + img_name))
    t1 = time.time()
    dlib_bounding_checks = dlib_detect_faces(img, cnn = CNN)
    t2 = time.time()
    aws_detected_faces = aws_detect_faces(BUCKET, S3_FOLDER + img_name, rekognition)
    t3 = time.time()
    
    dlib_times.append(t2-t1)
    aws_times.append(t3-t2)
    

    coordinates = [face['BoundingBox'] for face in aws_detected_faces['FaceDetails']]
    image_heigth = img.shape[0]
    image_width = img.shape[1]
    aws_bounding_checks = create_rectangle(coordinates, image_heigth, image_width)
    
    for d in aws_bounding_checks:
        cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), (0, 0, 255), 2)  
   
    if CNN: 
        for d in dlib_bounding_checks:
            cv2.rectangle(img, (d.rect.left(), d.rect.top()), (d.rect.right(), d.rect.bottom()), (137, 78, 0), 2)
    else:
        for d in dlib_bounding_checks:
            cv2.rectangle(img, (d.left(), d.top()), (d.right(), d.bottom()), (137, 78, 0), 2)   
    
    cv2.imwrite(SAVE_FOLDER + tag + img_name, img)
        
    # win.clear_overlay()
    # win.set_image(image)
    # win.add_overlay(dlib_bounding_checks, dlib.rgb_pixel(0,0,255))
    # win.add_overlay(aws_bounding_checks)

1. Processing image: Detection/2007_007763.jpg
2. Processing image: Detection/2008_001009.jpg
3. Processing image: Detection/2008_001322.jpg
4. Processing image: Detection/2008_002079.jpg
5. Processing image: Detection/2008_002470.jpg
6. Processing image: Detection/2008_002506.jpg
7. Processing image: Detection/2008_004176.jpg
8. Processing image: Detection/2008_007676.jpg
9. Processing image: Detection/2009_004587.jpg
10. Processing image: Detection/bald_guys.jpg
11. Processing image: Detection/cara_calvo.jpg
12. Processing image: Detection/cara_cara.jpg
13. Processing image: Detection/cara_perro.jpg
14. Processing image: Detection/dogs.jpg
15. Processing image: Detection/multitud.jpg
16. Processing image: Detection/selfie_oscar.jpg


In [67]:
f = open("Times/dlib_detection_times.txt","w") 
f.writelines('{}'.format(dlib_times))
f.close()

g = open("Times/aws_detection_times.txt","w") 
g.writelines('{}'.format(aws_times))
g.close()